In [36]:
import gradio as gr
import os
import time
import openai

def add_text(history, text):
    history.append((text, None))
    return history, gr.update(value="", interactive=False)

def bot(history):
    question = history[-1][0]

    messages = [{"role": "user", "content": question}]
    for h in history:
        if h[0] is not None:
            messages.append({"role": "user", "content": h[0]})
        if h[1] is not None:
            messages.append({"role": "assistant", "content": h[1]})

    gen = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True,
        temperature=0.5,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    history[-1][1] = ""

    while True:
        response = next(gen)
        delta = response.choices[0].delta
        if delta.content is not None:
            history[-1][1] += delta.content
            time.sleep(0.05)
            yield history
        else:
            break

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, os.path.join("./avatar.png")),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )

    txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    
    txt.submit(lambda: gr.update(interactive=True), None, [txt], queue=False)

demo.queue()
demo.launch(server_name='0.0.0.0') 

/home/hoon/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7896

To create a public link, set `share=True` in `launch()`.
